In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
import random
import os

In [ ]:
# Constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 224

## Testing the performance of the trained model

In [ ]:
def get_model():
    model = models.densenet201(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False  # Prevents the pre-trained weights from being updated during training.

    # Replace classifier
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 2)  # 2 classes: O, R
    )

    return model.to(device)

model = get_model()

In [ ]:
model = get_model()
model.load_state_dict("densenet201_final_checkpoint.pth", map_location=device)
model.eval()

In [ ]:
# Transforming the dataset using randomization 
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
def densenet_inference(image_input):
    """
    Returns 'O' or 'R' with confidence.
    """
    if isinstance(image_input, str):
        image = Image.open(image_input).convert("RGB")
    else:
        image = image_input.convert("RGB")

    # Apply test transformations
    img_tensor = test_transform(image).unsqueeze(0).to(device)
    densenet_class_names = ["O", "R"]

    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        output = model(img_tensor)  # model output
        probabilities = F.softmax(output, dim=1)  # all probabilities
        pred_class_idx = torch.argmax(
            probabilities, dim=1
        ).item()  # class index with highest probability
        confidence = probabilities[0, pred_class_idx].item()  # confidence score
        pred_class = densenet_class_names[pred_class_idx]  # class name

    print(f"[DenseNet] Prediction: {pred_class} (Confidence: {confidence:.2f})")
    return pred_class, confidence

In [ ]:
def get_image_paths(folder_path):
    """
    Returns a list of image file paths from the folder.
    """

    extensions = [".jpg", ".jpeg", ".png"]

    files = [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, f))
        and os.path.splitext(f)[1].lower() in extensions
    ]
    
    random.shuffle(files)

    return files

In [ ]:
files = get_image_paths(r'smart-dustbin-be-engine/ml/data')

for file in files:
    densenet_inference(file)